In [1]:
import time
import requests
import numpy as np
import pandas as pd
import logging
import datetime
from binance.um_futures import UMFutures
from binance.websocket.um_futures.websocket_client import UMFuturesWebsocketClient
from binance.lib.utils import config_logging

# where API KEY stored.(USE YOUR OWN KEY)
from cert import binanceKey

# my own library
from backTester.BackTester import BackTester
from backTester.BackDataLoader import BackDataLoader
from conditionGenerator.ConditionGenerator import ConditionGenerator

# visualization
import matplotlib.pyplot as plt
import seaborn as sns

In [12]:
# 현재 시간
timestamp_crit = time.mktime(datetime.datetime.today().timetuple())
timestamp_crit = int(timestamp_crit*1000)
print(timestamp_crit)

1671620773000


In [13]:
# data loading
client = UMFutures()
# backdata 가져오기 (간격 : 1m)
# 2022-08-21 18:30:00 ~ 2022-10-09 09:25:00
first_openTime = timestamp_crit
df1 = BackDataLoader().get_backdata(client, '1m', startTime=first_openTime, limit=1500)
df2 = BackDataLoader().get_backdata(client, '1m', startTime=first_openTime-5*1500*60*1000, limit=1500)
df3 = BackDataLoader().get_backdata(client, '1m', startTime=first_openTime-2*5*1500*60*1000, limit=1500)
df4 = BackDataLoader().get_backdata(client, '1m', startTime=first_openTime-3*5*1500*60*1000, limit=1500)
df5 = BackDataLoader().get_backdata(client, '1m', startTime=first_openTime-4*5*1500*60*1000, limit=1500)
df6 = BackDataLoader().get_backdata(client, '1m', startTime=first_openTime-5*5*1500*60*1000, limit=1500)
df7 = BackDataLoader().get_backdata(client, '1m', startTime=first_openTime-6*5*1500*60*1000, limit=1500)
df8 = BackDataLoader().get_backdata(client, '1m', startTime=first_openTime-7*5*1500*60*1000, limit=1500)
df9 = BackDataLoader().get_backdata(client, '1m', startTime=first_openTime-8*5*1500*60*1000, limit=1500)
df10 = BackDataLoader().get_backdata(client, '1m', startTime=first_openTime-9*5*1500*60*1000, limit=1500)

backdata = pd.concat([df10, df9, df8, df7, df6, df5, df4, df3, df2, df1], axis=0)
backdata.reset_index(inplace=True, drop=True)

,OpenTime,Open,High,Low,Close,Volume,CloseTime,QuoteAssetVolume,NumOfTrades,TakerBuyBaseAssetVolume,TakerBuyQuoteAssetVolume
0,2022-11-04 14:07:00,20775.6,20781.9,20751.1,20760.4,455.439,2022-11-04 14:07:59.999,9.455816e+06,3025,152.514,3.167104e+06
1,2022-11-04 14:08:00,20760.3,20764.7,20750.3,20757.4,454.387,2022-11-04 14:08:59.999,9.432000e+06,2726,156.834,3.255377e+06
2,2022-11-04 14:09:00,20757.4,20788.5,20757.4,20781.3,497.791,2022-11-04 14:09:59.999,1.034198e+07,2829,345.962,7.187096e+06
3,2022-11-04 14:10:00,20781.2,20790.0,20770.1,20780.4,579.919,2022-11-04 14:10:59.999,1.205151e+07,3099,286.877,5.961562e+06
4,2022-11-04 14:11:00,20780.4,20817.7,20780.4,20808.1,1490.085,2022-11-04 14:11:59.999,3.100242e+07,7185,1037.942,2.159361e+07


In [6]:
# TESTNET을 활용해서...
from cert import binanceKey

testBaseUrl_rest = "https://testnet.binancefuture.com"
testBaseUrl_wsk = "wss://stream.binancefuture.com"

In [9]:
import asyncio
import websockets

async def connect():
    API_KEY = binanceKey.API_KEY
    SECRET_KEY = binanceKey.SECRET_KEY

    url = testBaseUrl_wsk
    btcusdt_streamName = dict()
    btcusdt_streamName['aggTrade'] = "btcusdt@aggTrade"
    btcusdt_streamName['depth'] = "btcusdt@depth"
    
    async with websockets.connect(testBaseUrl_wsk) as websocket:

        while True:
            flag = int(input("1. Continue, 0. Exit :\n"))

            if flag==1:
                strNm_AT = btcusdt_streamName['aggTrade']
                strNm_DT = btcusdt_streamName['depth']

                senddata = {
                    "method":"SUBSCRIBE",
                    "params":[
                        strNm_AT,
                        strNm_DT,
                    ],
                    "id":1
                }

                await websocket.send(senddata)
                while True:
                    data = await websocket.recv()
                    print("Recev Command is :", data)

            elif flag==0:
                print("Exit!")
                break
            else:
                print("Invalid input.")
                continue

co = connect()
await co
co.close()

ConnectionClosedError: no close frame received or sent